# 多数決方式のテンプレート

- GAOZの組みを全く変えていない場合に利用する
  -> 多数決方式でせめる  

### 派生方式

#### election with ignorance  

多数決をするが、候補が複数あり、合致しているものを採用する方式。  

ただし、NaNの数が多くなるので、ハミング距離が必然的に小さくなり、一見良さそうに見えるのに注意

#### wegihted election

多数決をする際に、Reviewのカラムの長さが長い順に重みを大きくして、多数決を行う。

MAEの性質上、カラム数が多い方が実際の値に全体的に近くなることが考えられるからである。

### 担当者と日付
2024/10/☆☆  WHO

In [11]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [12]:
# fmt: off
ALL_HEADER = ['Name', 'Gender', 'Age', 'Occupation', 'ZIP-code', '2', '56', '247', '260', '653', '673', '810', '885',  # noqa: E501
                      '1009', '1073', '1097', '1126', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017',  # noqa: E501
                      '2021', '2043', '2086', '2087', '2093', '2100', '2105', '2138', '2143', '2174', '2193', '2253',  # noqa: E501
                      '2399', '2628', '2797', '2872', '2968', '3393', '3438', '3439', '3440', '3466', '3479', '3489',  # noqa: E501
                      '3877', '3889'] # noqa

B_HEADERS_LIST = [
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '260', '653', '1525', '2105', '2193', '2253', '2628', '2872', '3438', '3439', '3440', '3877', '3889'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '2', '56', '260', '653', '673', '1009', '1073', '1525', '1750', '1881', '1967', '2043', '2093', '2105', '2143', '2193', '2399', '2628', '2968', '3479', '3489', '3877', '3889'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '673', '1881', '1920', '2087', '2138'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '2', '56', '673', '810', '885', '1009', '1073', '1097', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017', '2043', '2087', '2093', '2138', '2399', '3438', '3439', '3440'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '673', '810', '1073', '1126', '1702', '2100', '2174', '2253', '2797', '3393', '3466'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '247', '885', '1097', '1654', '2086', '2138', '2872'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '247', '2100', '2143', '2872', '3479'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '260', '1097', '1750', '2021', '2093', '2105', '2628', '2968'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '247', '1920', '2017', '2087'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '260', '1097', '2628', '2174', '2797', '1073', '2100', '2968', '2105', '2193'],  # noqa: E501
]

B_USER_ATTRIBUTE_HEADERS = ['Gender', 'Age', 'Occupation', 'ZIP-code']

B_REVIEW_HEADERS_LIST = [
  ['260', '653', '1525', '2105', '2193', '2253', '2628', '2872', '3438', '3439', '3440', '3877', '3889'],  # noqa: E501
  ['2', '56', '260', '653', '673', '1009', '1073', '1525', '1750', '1881', '1967', '2043', '2093', '2105', '2143', '2193', '2399', '2628', '2968', '3479', '3489', '3877', '3889'],  # noqa: E501
  ['673', '1881', '1920', '2087', '2138'],  # noqa: E501
  ['2', '56', '673', '810', '885', '1009', '1073', '1097', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017', '2043', '2087', '2093', '2138', '2399', '3438', '3439', '3440'],  # noqa: E501
  ['673', '810', '1073', '1126', '1702', '2100', '2174', '2253', '2797', '3393', '3466'],  # noqa: E501
  ['247', '885', '1097', '1654', '2086', '2138', '2872'],  # noqa: E501
  ['247', '2100', '2143', '2872', '3479'],  # noqa: E501
  ['260', '1097', '1750', '2021', '2093', '2105', '2628', '2968'],  # noqa: E501
  ['247', '1920', '2017', '2087'],  # noqa: E501
  ['260', '1097', '2628', '2174', '2797', '1073', '2100', '2968', '2105', '2193'],  # noqa: E501
]

MOVIE_IDS = ['2', '56', '247', '260', '653', '673', '810', '885', '1009', '1073', '1097', '1126', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017', '2021', '2043', '2086', '2087', '2093', '2100', '2105', '2138', '2143', '2174', '2193', '2253', '2399', '2628', '2797', '2872', '2968', '3393', '3438', '3439', '3440', '3466', '3479', '3489', '3877', '3889']
# fmt: on


### C_0 ~ C_9までの読み込み

In [13]:
from os import listdir

ATTACK_TARGET = "08"

c_data_list = []

files = listdir("../data")

for i in range(10):
    c_data = pd.read_csv(f"../data/C{ATTACK_TARGET}_{i}.csv")
    file = None
    for f in files:
        if f.startswith(f"C{ATTACK_TARGET}_{i}"):
            file = f
            break
    if file is None:
        print(f)
        raise("File not found")
    c_data_list.append(c_data)

c_data_list[0]

,Gender,Age,Occupation,ZIP-code,260,653,1525,2105,2193,2253,2628,2872,3438,3439,3440,3877,3889
0,M,56,6,229,2,3,0,5,2,3,2,3,2,4,2,2,1
1,M,35,11,838,0,5,3,1,2,0,2,1,3,0,1,0,3
2,F,56,3,751,0,0,0,0,0,3,4,4,0,0,0,3,0
3,F,45,17,459,2,2,4,4,2,3,4,1,1,0,1,0,2
4,F,45,1,114,1,4,5,3,1,0,1,5,4,1,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,M,25,3,440,5,4,0,0,0,0,2,2,1,0,0,0,0
9996,M,25,6,436,0,0,0,0,0,0,0,0,4,0,0,0,0
9997,M,50,3,320,0,0,3,0,1,0,4,5,1,2,1,0,5
9998,M,56,14,765,1,0,3,3,0,2,2,4,0,3,2,1,0


### Ba, Bbの読み込み

In [14]:
Ba = pd.read_csv(f"../data/B{ATTACK_TARGET}a.csv", names=ALL_HEADER)
Ba = Ba.drop(0)
Ba.head()

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
1,Mitch Willshere,M,56,0,977,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pippo Pordall,M,56,3,535,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Ric MacGuffie,M,45,16,327,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dionis Ginity,F,25,9,463,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Constancy Swainger,F,50,14,454,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
Bb = pd.read_csv(f"../data/B{ATTACK_TARGET}b.csv")
Bb.head()

,2,56,247,260,653,673,810,885,1009,1073,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
0,3,0,0,2,4,0,3,0,4,*,...,2,1,4,0,5,3,1,1,3,5
1,0,0,3,2,5,0,0,0,0,2,...,4,0,1,0,2,0,0,0,0,4
2,0,4,5,5,5,0,0,0,0,2,...,3,1,0,0,0,2,3,3,0,0
3,4,1,5,3,2,4,2,0,*,3,...,0,0,4,0,4,5,4,5,5,5
4,4,0,0,2,0,3,4,2,2,1,...,4,1,2,3,2,0,0,2,0,0


### BaのN番目の行と完全一致する行をC0~C9から取得

In [16]:
# Baの2行目を選択
row_index = 2

gender = Ba.iloc[row_index]["Gender"]
age = Ba.iloc[row_index]["Age"]
occupation = Ba.iloc[row_index]["Occupation"]
zipcode = Ba.iloc[row_index]["ZIP-code"]

print("検索条件")
print(gender, age, occupation, zipcode)

filtered_C_list = []

# 完全一致を見つける
for c in c_data_list:
    filtered_C_0 = c[(c['Gender'] == gender) & 
                      (c['Age'] == int(age)) & 
                      (c['Occupation'] == int(occupation)) & 
                      (c['ZIP-code'] == int(zipcode))]
    filtered_C_list.append(filtered_C_0)

print("完全一致した件数 (10であることが望まれる)")
print(len(filtered_C_list))

検索条件
M 45 16 327
完全一致した件数 (10であることが望まれる)
10


### 各カラムごとに多数決の候補を表示する

In [17]:
merged_rows = []

for i, filtered_df in enumerate(filtered_C_list):
    if not filtered_df.empty:
        # 各フィルタリングされたデータフレームから1行だけ取得
        row = filtered_df.iloc[0]
        
        # ALL_HEADERに基づいて新しい行を作成
        new_row = pd.Series(index=ALL_HEADER)
        
        # データを転送
        for col in ALL_HEADER:
            if col in filtered_df.columns:
                new_row[col] = row[col]
            else:
                new_row[col] = np.nan
        
        # 新しい行をリストに追加
        merged_rows.append(new_row)

# リストからデータフレームを作成
merged_df = pd.DataFrame(merged_rows, columns=ALL_HEADER)
merged_df

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
0,NaN,M,45,16,327,0,2,NaN,NaN,NaN,...,NaN,NaN,3,1,1,NaN,NaN,NaN,NaN,NaN


### ざっと見て、多数決の候補にばらつきはなさそうと判断し、各行に対してランダムに一つ値を取得できるようにする

In [18]:
def select_random_value(column):
    # NaNを除外した値のリストを作成
    valid_values = column.dropna().tolist()
    if valid_values:
        # 有効な値がある場合、ランダムに1つ選択
        return np.random.choice(valid_values)
    else:
        # 有効な値がない場合はNaNを返す
        return np.nan

# 各カラムからランダムに値を選択
random_values = {}
for column in merged_df.columns:
    random_values[column] = select_random_value(merged_df[column])

# 結果を新しいデータフレームとして作成
result_df = pd.DataFrame([random_values])
result_df

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
0,NaN,M,45,16,327,0,2,NaN,NaN,NaN,...,NaN,NaN,3,1,1,NaN,NaN,NaN,NaN,NaN


### 以上の処理をBaの各行（0~49行）に対して実行する

In [19]:
import pandas as pd
import numpy as np

final_df = pd.DataFrame(columns=ALL_HEADER)

for row_index in range(50):
    # Baデータフレームから現在の行の情報を取得
    gender = Ba.iloc[row_index]["Gender"]
    age = Ba.iloc[row_index]["Age"]
    occupation = Ba.iloc[row_index]["Occupation"]
    zipcode = Ba.iloc[row_index]["ZIP-code"]

    filtered_C_list = []

    # 各cデータフレームに対して、完全一致するレコードをフィルタリング
    for c in c_data_list:
        filtered_C_0 = c[(c['Gender'] == gender) & 
                        (c['Age'] == int(age)) & 
                        (c['Occupation'] == int(occupation)) & 
                        (c['ZIP-code'] == int(zipcode))]
        filtered_C_list.append(filtered_C_0)

    # 多数決の候補となる行のリストを作成
    candidate_rows = []
    for i, filtered_df in enumerate(filtered_C_list):
        if not filtered_df.empty:
            # 各フィルタリングされたデータフレームから1行だけ取得
            row = filtered_df.iloc[0]
            # ALL_HEADERに基づいて新しい行を作成し、データを転送
            candidate_row = pd.Series(index=ALL_HEADER)
            for col in ALL_HEADER:
                candidate_row[col] = row[col] if col in filtered_df.columns else np.nan
            # 新しい行を候補リストに追加
            candidate_rows.append(candidate_row)

    # 候補行のリストからデータフレームを作成
    candidate_rows_df = pd.DataFrame(candidate_rows, columns=ALL_HEADER)

    # 各列に対して、ランダムに値を選択（多数決の実装）
    elected_value = {}
    for column in ALL_HEADER:
        elected_value[column] = select_random_value(candidate_rows_df[column])

    # 選択された値から新しい行を作成
    elected_value_row_df = pd.DataFrame([elected_value])

    # 選択された行をfinal_dfの最後に追加
    final_df = pd.concat([final_df, elected_value_row_df], ignore_index=True)

final_df

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
0,NaN,M,56,0,977,4.0,0.0,0.0,3.0,NaN,...,3.0,0.0,4.0,5.0,0.0,0.0,5.0,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,M,45,16,327,0.0,2.0,NaN,NaN,NaN,...,NaN,NaN,3.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
3,NaN,F,25,9,463,2.0,0.0,5.0,0.0,0.0,...,4.0,NaN,0.0,3.0,2.0,NaN,0.0,3.0,0.0,1.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,M,18,9,764,0.0,0.0,0.0,5.0,5.0,...,4.0,NaN,0.0,0.0,2.0,NaN,0.0,0.0,0.0,2.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 最終的に得た映画のレビューの候補とBbを比較し、Bbの何番目のindexが一番近くてハミング距離にどれくらいの違いがあるかを取得する

In [20]:

def hamming_distance(a, b):
    a_int = a.fillna(0).astype(int)
    b_int = b.replace('*', None).fillna(0).astype(int)
    # ハミング距離の計算
    return np.sum(a_int != b_int)

def find_closest_row(row, Bb):
    distances = Bb.apply(lambda bb_row: hamming_distance(row[MOVIE_IDS], bb_row[MOVIE_IDS]), axis=1)
    min_distance = distances.min()
    closest_index = distances.idxmin()
    return pd.Series({'Min_Hamming_Distance': min_distance, 'Closest_Bb_Index': closest_index})

# 結果を格納するデータフレーム
result_df = pd.DataFrame(index=final_df.index, columns=['Min_Hamming_Distance', 'Closest_Bb_Index'])

# 各行に対してハミング距離を計算し、最も近い行を見つける
for idx, row in final_df[MOVIE_IDS].iterrows():
    result = find_closest_row(row, Bb[MOVIE_IDS])
    result_df.loc[idx] = result

# 元のfinal_dfにハミング距離の結果を結合
final_result_df = pd.concat([final_df, result_df], axis=1)
final_result_df

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,3438,3439,3440,3466,3479,3489,3877,3889,Min_Hamming_Distance,Closest_Bb_Index
0,NaN,M,56,0,977,4.0,0.0,0.0,3.0,NaN,...,4.0,5.0,0.0,0.0,5.0,NaN,NaN,NaN,26,36
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,35
2,NaN,M,45,16,327,0.0,2.0,NaN,NaN,NaN,...,3.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,21,38
3,NaN,F,25,9,463,2.0,0.0,5.0,0.0,0.0,...,0.0,3.0,2.0,NaN,0.0,3.0,0.0,1.0,17,43
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,35
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,35
6,NaN,M,18,9,764,0.0,0.0,0.0,5.0,5.0,...,0.0,0.0,2.0,NaN,0.0,0.0,0.0,2.0,19,27
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,35
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,35
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,35
